Programmatic way to generate config files for a given hyperparameter sweep.

In [1]:
import os
import yaml
import itertools
import numpy as np

In [2]:
# Specify a directory for experiment

sweep_experiment_name = "sweep_whales_may11_s4"
sweep_experiment_dir_parent = "/home/jupyter"
sweep_experiment_dir = os.path.join(sweep_experiment_dir_parent, sweep_experiment_name)

if not os.path.exists(sweep_experiment_dir):
    os.makedirs(sweep_experiment_dir)

In [3]:
# Specify model type

model_type = 'supervised_nn'

In [4]:
# workhorse to generate combinations of parameters

def generate_choice_combinations(d):
    # d is a dict of lists
    # require that the keys of d are all of the same type (eg strings)
    # output a dict {i:{key:value} for i in range(N)}
    # which contains all possible combinations of values (encoded in the lists)
    # and N = product of len(d[key]), for all keys in d
    sorted_keys = sorted(d.keys())
    choices_lex = []
    num_choices = {}
    for key in sorted_keys:
        num_choices[key] = len(d[key])
        choices_lex.append(list(range(num_choices[key])))
    choices_cartesian = list(itertools.product(*choices_lex))
    configs_cartesian = {}
    for i, choices in enumerate(choices_cartesian):
        configs_cartesian[i] = {sorted_keys[j]: d[sorted_keys[j]][choices[j]] for j in range(len(sorted_keys))}
        
    return configs_cartesian

In [5]:
# Specify general parameters

sweep_config = {}
sweep_config['input_vars'] = [['AccX',
                               'AccY',
                               'AccZ',
                               'StaticAccX',
                               'StaticAccY',
                               'StaticAccZ',
                               'Depth', 
                               'Speed']]


# Dogs: 
# sweep_config['input_vars'] = [['AccX_Back',
#                                'AccY_Back',
#                                'AccZ_Back', 
#                                'AccX_Neck', 
#                                'AccY_Neck', 
#                                'AccZ_Neck', 
#                                'StaticAccX_Back', 
#                                'StaticAccY_Back', 
#                                'StaticAccZ_Back', 
#                                'StaticAccX_Neck',
#                                'StaticAccY_Neck', 
#                                'StaticAccZ_Neck',
#                                'GyrX_Back', 
#                                'GyrY_Back', 
#                                'GyrZ_Back', 
#                                'GyrX_Neck', 
#                                'GyrY_Neck', 
#                                'GyrZ_Neck']]
sweep_config['dataset_dir'] = ['/home/jupyter/behavior_data_local/data/formatted/friedlaender_whales']
sweep_config['model'] = [model_type] # do not change
sweep_config['output_parent_dir'] = [sweep_experiment_dir]
sweep_config['num_clusters'] = [5]
sweep_config['read_latents'] = [False] # should be either [True] or [False]
#sweep_config['data_latents_fp_glob'] = [['/home/jupyter/behavior_benchmarks_outputs/vehkaoja_dogs/whiten_dogs/latents/*.npy']]

summary = sweep_config.copy()

In [6]:
# Specify model-specific parameters

if model_type == 'kmeans':
    sweep_model_config = {}
    
    summary['kmeans_config'] = sweep_model_config

if model_type == 'eskmeans':
    sweep_model_config = {}
    sweep_model_config['boundary_init_lambda'] = [15.0]
    sweep_model_config['embed_length'] = [20]
    sweep_model_config['landmark_hop_size'] = [10]
    sweep_model_config['n_epochs'] = [10]
    sweep_model_config['n_landmarks_max'] = [40]
    sweep_model_config['batch_size'] = [1]
    sweep_model_config['time_power_term'] = [.9, .85]
    
    summary['eskmeans_config'] = sweep_model_config
    
if model_type == 'vame':
    sweep_model_config = {}
    sweep_model_config['batch_size'] = [1024]
    sweep_model_config['max_epochs'] = [100]
    sweep_model_config['beta'] = [0] ## Scalar multiplied by KL loss
    sweep_model_config['zdims'] = [5, 10, 30] ## Latent space dimensionality
    sweep_model_config['learning_rate'] = [0.0005]
    sweep_model_config['time_window_sec'] = [2, 5]
    sweep_model_config['prediction_decoder'] = [1] ## Whether to predict future steps
    sweep_model_config['prediction_sec'] = [2, 5] ## How much to predict after encoded window
    sweep_model_config['scheduler'] = [1]
    sweep_model_config['scheduler_step_size'] = [100]
    sweep_model_config['scheduler_gamma'] = [0.2]
    sweep_model_config['kmeans_lambda'] = [0.1]
    sweep_model_config['downsizing_factor'] = [8]
    
    summary['vame_config'] = sweep_model_config
    
if model_type == 'hmm':
    sweep_model_config = {}
    sweep_model_config['time_bins'] = [2500] 
    sweep_model_config['prior_wait_sec'] = [5., 10.] 
    sweep_model_config['sticky_prior_strength'] = [0., 1., 2.]
    sweep_model_config['N_iters'] = [50]
    sweep_model_config['lags'] = [1, 2]
        
    summary['vame_config'] = sweep_model_config
    
if model_type == 'supervised_nn':
    sweep_model_config = {}
    sweep_model_config['downsizing_factor'] = [2048]
    sweep_model_config['lr'] = [1e-2, 1e-3]
    sweep_model_config['weight_decay'] = [5e-3, 1e-3, 1e-4]
    sweep_model_config['n_epochs'] = [150]
    sweep_model_config['hidden_size'] = [8, 16, 32]
    sweep_model_config['num_layers'] = [4, 8, 16]
    sweep_model_config['temporal_window_samples'] = [8192] # used for training only, to subselect
    sweep_model_config['batch_size'] = [8]
    sweep_model_config['dropout'] = [0]
    sweep_model_config['rescale_param'] = [0]
    sweep_model_config['jitter_scale'] = [0, 0.02]
    sweep_model_config['blur_scale'] = [0, 0.02]
    sweep_model_config['weight_factor'] = [0.5]
    sweep_model_config['sparse_annotations'] = [False]
    
    summary['supervised_nn_config'] = sweep_model_config
    
if model_type == 'vq_cpc':
    sweep_model_config = {'predict_proportion' : [0.5], 
                          'encoder_kernel_width' : [7],
                          'downsizing_factor' : [256], 
                          'lr' : [4e-4], 
                          'n_epochs' : [300], 
                          'conv_stack_hidden_size' : [64], 
                          'temporal_window_samples' : [1024],
                          'batch_size' : [64], 
                          'conv_stack_depth' : [12], 
                          'z_dim' : [8], 
                          'c_dim' : [16], 
                          'warmup_epochs' : [150], 
                          'initial_lr' : [1e-5],
                          'blur_scale' : [0.], 
                          'jitter_scale' : [0.],
                          'pooling_factor' : [32]}
        
    summary['vq_cpc_config'] = sweep_model_config

In [7]:
# Specify evaluation parameters
sweep_evaluation_config = {}
sweep_evaluation_config['boundary_tolerance_sec'] = [1.]

summary['evaluation'] = sweep_evaluation_config

In [8]:
# Summarize what the experiment is about and save off

summary['summary'] = "sweep simple gru whales"
target_filename = sweep_experiment_name + "_summary" + '.yaml'
target_fp = os.path.join(sweep_experiment_dir, target_filename)                       
with open(target_fp, 'w') as file:
    yaml.dump(summary, file)

In [9]:
# Make cartesian combinations for sub-dictionaries
sweep_evaluation_cartesian = generate_choice_combinations(sweep_evaluation_config)
sweep_model_cartesian = generate_choice_combinations(sweep_model_config)

In [10]:
# Incorporate into main sweep dict

model_config_key_name = model_type + "_config"
sweep_config[model_config_key_name] = [sweep_model_cartesian[key] for key in sweep_model_cartesian]

sweep_config['evaluation'] = [sweep_evaluation_cartesian[key] for key in sweep_evaluation_cartesian]

In [11]:
sweep_config_cartesian = generate_choice_combinations(sweep_config)

In [12]:
# number so as to get experiment names
for i in sweep_config_cartesian.keys():
    experiment_name = sweep_experiment_name + "_" + str(i)
    sweep_config_cartesian[i]['experiment_name'] = experiment_name

In [13]:
# save off configs:

config_fps = []
for i in sweep_config_cartesian.keys():
    config = sweep_config_cartesian[i]
    target_filename = config['experiment_name'] + '.yaml'
    target_fp = os.path.join(sweep_experiment_dir, target_filename)
    config_fps.append(target_fp)                         
    with open(target_fp, 'w') as file:
        yaml.dump(config, file)

In [14]:
# random search?

randomize = True

if randomize:
    config_fps = np.random.permutation(config_fps)

In [15]:
# Generate command line prompt to run
output = ""
for config_fp in config_fps:
    output += "python full_experiment.py --config " + config_fp + "; "

print(output)
#python train_model.py --config /home/jupyter/behavior_benchmarks/behavior_benchmarks/example_config/example_config_eskmeans.yaml

python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_141.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_194.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_104.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_116.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_168.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_55.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_170.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_4.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep_whales_may11_s4_124.yaml; python full_experiment.py --config /home/jupyter/sweep_whales_may11_s4/sweep